# lookup_nutrients

In [61]:
import pandas as pd
import sqlite3
from rapidfuzz import process, fuzz, utils

Actually, the next thing I want to do is make a python script that will give you nutritional info about any food by searching the most similar item in the table and getting that info. I want to also add in a chatbot that can tell if there are multiple ingredients to give a better guess and estimate the price.

In [50]:
# Access the database

conn = sqlite3.connect("food_data.db")
cur = conn.cursor()

In [7]:
pd.read_sql("SELECT * FROM food_table LIMIT 5;", conn)

,food_nutrient_core_id,fdc_id,description,calories,protein_g,carbs_g,fat_g,fiber_g,sodium_mg,penalty
0,1,319877,hummus,225.0668,7.287500,13.5667,16.9500,5.45,438.3636,0
1,5,320027,milk 2,30.2813,3.282500,NaN,1.9057,NaN,38.9167,2
2,9,320381,minerals,NaN,NaN,NaN,NaN,NaN,53.8571,5
3,13,320413,tomatoes grape,13.1872,0.833125,2.1000,0.6283,2.10,5.8333,0
4,14,321359,milk reduced fat fluid 2 milkfat with added vi...,50.0000,3.360000,4.9000,1.9000,NaN,39.0000,1


In [55]:
choices = pd.read_sql("""
    SELECT description 
    FROM food_table;
""", conn)["description"].dropna().tolist()

def find_match(x):
    matches = process.extract(x, choices, scorer=fuzz.WRatio, limit=2, processor=utils.default_process)
    
    matches_dict = {}
    for name, score, idx in matches:
        matches_dict[name] = pd.read_sql(f"SELECT penalty FROM food_table WHERE description = ?;", conn, params=(name,))['penalty'].tolist()[0]

    best_match = min(matches_dict, key=matches_dict.get)

    best_match_nutri = pd.read_sql("""
        SELECT description, calories, protein_g, carbs_g, fat_g, fiber_g, sodium_mg 
        FROM food_table 
        WHERE description = ?;
    """, conn, params=(best_match,)).iloc[[0]].iloc[0].tolist()

    best_match_nutri_dict = {
        'description': best_match_nutri[0], 
        'calories': best_match_nutri[1], 
        'protein_g': best_match_nutri[2], 
        'carbs_g': best_match_nutri[3],
        'fat_g': best_match_nutri[4],
        'fiber_g': best_match_nutri[5],
        'sodium_mg': best_match_nutri[6]
    }

    return best_match_nutri_dict

In [60]:
x = input("Food Item:")
x_match = find_match(x)
x_match

Food Item: milk half and half


{'description': 'milk reduced fat fluid 2 milkfat with added vitamin a and vitamin d',
 'calories': 50.0,
 'protein_g': 3.36,
 'carbs_g': 4.9,
 'fat_g': 1.9,
 'fiber_g': None,
 'sodium_mg': 39.0}